<h1>Corresponsales Bancarios</h1>

<h3>0. Imports and Parameters</h3>

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 9999

In [2]:
SEP_FILES = ";"
# Tipos de Operacion
RETIROS_TYPE = ["RETIRO_OTP", "RETIROCB"]
RECAUDOS_TYPE = ["RECAUDOS", "RECAUDO"]
PAGOS_TYPE = ["PAGO_PRESTAMOS", "PAGO_TARJETA_CREDITO"]
DEPOSITOS_TYPE = ["DEPOSITOS"]

# Estados Operacion
SUCCESS_ESTATE = ['RETIRO OTP EXITOSO','RECAUDO EXITOSO','REVERSO RECAUDO EXITOSO','PAGO PRESTAMOS EXITOSO','DEPOSITO EXITOSO','PAGO TARCRED EXITOSO', 'RETIROCBPAGADO', 'CARGOGENERADO','REVERSO RETIRO OTP EXITOSO']
FAIL_ESTATE = ['RETIRO OTP NO EXITOSO', 'PAGO PRESTAMOS NO EXITOSO', 'PAGO TARCRED NO EXITOSO','PENDIENTEVALIDACION','TRANS-FAIL-C','CARGOFALLIDO', 'ANULADA']
OTHER_ESTATE = ['FACTURAVALIDADA','RETIROCBVALIDO']

<h3>1. Lectura de Insumos</h3>

In [3]:
new_format_files = [
    "/Users/alfonsocaro/developer/code/corresponsales_etl/src/input/LOG_TRAN_CBSWS_F220225.txt"
]

old_format_files = [
    "/Users/alfonsocaro/developer/code/corresponsales_etl/src/input/logcb_20220227.csv"
]

In [4]:
li = []
for filename in new_format_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep=SEP_FILES)
    li.append(df)

new_format_frame = pd.concat(li, axis=0, ignore_index=True)

In [5]:
li = []
for filename in old_format_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep=SEP_FILES)
    li.append(df)

old_format_frame = pd.concat(li, axis=0, ignore_index=True)

<h3>2. Limpieza y Union de Tablas</h3>

In [6]:
new_format_frame["FECHA_CADUC"] = np.nan                                                                                                                            
old_format_frame.rename(columns = {'FECHA_CON':'FECHA_CONTA', 'NUMERO_CUENTA':'NUM_CUENTA'}, inplace = True)
new_format_frame = new_format_frame.reindex(columns=[
     'CB',
     'UPC',
     'TIPO',
     'MEDIO',
     'ESTADO',
     'FECHA_OPER',
     'FECHA_CONTA',
     'VALOR',
     'FECHA_CREA',
     'FECHA_CADUC',
     'FACTURA',
     'TIPO_CUENTA',
     'NUM_CUENTA',
     'CONVENIO',
     'MOVIMIENTO',
     'MOVIMIENTO_2',
     'CORRESPONSAL',
     'PUNTO',
     'TERMINAL',
     'DESCRIP'
])

In [7]:
frame = pd.concat([new_format_frame, old_format_frame], axis=0, ignore_index=True)

In [8]:
frame['TIPO'] = frame['TIPO'].str.strip()
frame['ESTADO'] = frame['ESTADO'].str.strip()
frame['CORRESPONSAL'] = frame['CORRESPONSAL'].str.strip()
frame['VALOR'] = frame['VALOR'].str.strip()
frame['VALOR'] = frame['VALOR'].str.replace(',','.')

<h3>3. Totales</h3>

In [9]:
t_success  = frame[frame['ESTADO'].isin(SUCCESS_ESTATE)].shape[0]
t_failure  = frame[frame['ESTADO'].isin(FAIL_ESTATE)].shape[0]
t_others   = frame[~frame['ESTADO'].isin((SUCCESS_ESTATE + FAIL_ESTATE))].shape[0]

In [10]:
print(f"Total Exitosas: {t_success}")
print(f"Total Fallidas: {t_failure}")
print(f"Total Otras: {t_others}")

Total Exitosas: 25134
Total Fallidas: 559
Total Otras: 89


In [11]:
retiros_f = frame[frame['TIPO'].isin(RETIROS_TYPE)]
recaudos_f = frame[frame['TIPO'].isin(RECAUDOS_TYPE)]
pagos_f = frame[frame['TIPO'].isin(PAGOS_TYPE)]
depositos_f = frame[frame['TIPO'].isin(DEPOSITOS_TYPE)]
otros_f = frame[~frame['TIPO'].isin((RETIROS_TYPE + RECAUDOS_TYPE + PAGOS_TYPE + DEPOSITOS_TYPE))]

<h3>a. Retiros</h3>

In [12]:
# retiros
t_retiros = retiros_f.shape[0]
t_suss_retiros  = retiros_f[retiros_f['ESTADO'].isin(SUCCESS_ESTATE)].shape[0]
t_fail_retiros  = retiros_f[retiros_f['ESTADO'].isin(FAIL_ESTATE)].shape[0]
t_other_retiros = retiros_f[~retiros_f['ESTADO'].isin((SUCCESS_ESTATE + FAIL_ESTATE))].shape[0]

In [13]:
print(f"Total retiros: {t_retiros}")
print(f"Total retiros exitosas: {t_suss_retiros}")
print(f"Total retiros fallidos: {t_fail_retiros}")
print(f"Total retiros otros: {t_other_retiros}")

Total retiros: 5759
Total retiros exitosas: 5570
Total retiros fallidos: 159
Total retiros otros: 30


<h3>b. Recaudos</h3>

In [14]:
# recaudos
t_recaudos = recaudos_f.shape[0]
t_suss_recaudos  = recaudos_f[recaudos_f['ESTADO'].isin(SUCCESS_ESTATE)].shape[0]
t_fail_recaudos  = recaudos_f[recaudos_f['ESTADO'].isin(FAIL_ESTATE)].shape[0]
t_other_recaudos = recaudos_f[~recaudos_f['ESTADO'].isin((SUCCESS_ESTATE + FAIL_ESTATE))].shape[0]

In [15]:
print(f"Total recaudos: {t_recaudos}")
print(f"Total recaudos exitosas: {t_suss_recaudos}")
print(f"Total recaudos fallidos: {t_fail_recaudos}")
print(f"Total recaudos otros: {t_other_recaudos}")

Total recaudos: 17670
Total recaudos exitosas: 17220
Total recaudos fallidos: 393
Total recaudos otros: 57


<h3>c. Pagos</h3>

In [16]:
# pagos
t_pagos = pagos_f.shape[0]
t_suss_pagos  = pagos_f[pagos_f['ESTADO'].isin(SUCCESS_ESTATE)].shape[0]
t_fail_pagos  = pagos_f[pagos_f['ESTADO'].isin(FAIL_ESTATE)].shape[0]
t_other_pagos = pagos_f[~pagos_f['ESTADO'].isin((SUCCESS_ESTATE + FAIL_ESTATE))].shape[0]

In [17]:
print(f"Total pagos: {t_pagos}")
print(f"Total pagos exitosas: {t_suss_pagos}")
print(f"Total pagos fallidos: {t_fail_pagos}")
print(f"Total pagos otros: {t_other_pagos}")

Total pagos: 842
Total pagos exitosas: 835
Total pagos fallidos: 7
Total pagos otros: 0


<h3>d. Depositos</h3>

In [18]:
# depositos
t_depositos = depositos_f.shape[0]
t_suss_depositos  = depositos_f[depositos_f['ESTADO'].isin(SUCCESS_ESTATE)].shape[0]
t_fail_depositos  = depositos_f[depositos_f['ESTADO'].isin(FAIL_ESTATE)].shape[0]
t_other_depositos = depositos_f[~depositos_f['ESTADO'].isin((SUCCESS_ESTATE + FAIL_ESTATE))].shape[0]

In [19]:
print(f"Total depositos: {t_depositos}")
print(f"Total depositos exitosas: {t_suss_depositos}")
print(f"Total depositos fallidos: {t_fail_depositos}")
print(f"Total depositos otros: {t_other_depositos}")

Total depositos: 1509
Total depositos exitosas: 1509
Total depositos fallidos: 0
Total depositos otros: 0


<h3>e. Otros</h3>

In [20]:
# otros
t_otros = otros_f.shape[0]
t_suss_otros  = otros_f[otros_f['ESTADO'].isin(SUCCESS_ESTATE)].shape[0]
t_fail_otros  = otros_f[otros_f['ESTADO'].isin(FAIL_ESTATE)].shape[0]
t_other_otros = otros_f[~otros_f['ESTADO'].isin((SUCCESS_ESTATE + FAIL_ESTATE))].shape[0]

In [21]:
print(f"Total otros: {t_otros}")
print(f"Total otros exitosas: {t_suss_otros}")
print(f"Total otros fallidos: {t_fail_otros}")
print(f"Total otros otros: {t_other_otros}")

Total otros: 2
Total otros exitosas: 0
Total otros fallidos: 0
Total otros otros: 2


In [22]:
frame.head()

,CB,UPC,TIPO,MEDIO,ESTADO,FECHA_OPER,FECHA_CONTA,VALOR,FECHA_CREA,FECHA_CADUC,FACTURA,TIPO_CUENTA,NUM_CUENTA,CONVENIO,MOVIMIENTO,MOVIMIENTO_2,CORRESPONSAL,PUNTO,TERMINAL,DESCRIP
0,4,A0007800370,RETIRO_OTP,EF,RETIRO OTP EXITOSO,24/02/2022 21:35,NaN,700000.00,24/02/2022 21:35,NaN,,CC,NaN,,,,GELSA-PAGA-TODO,9924070.0,9924070.0,NaN
1,4,A0007800371,RETIRO_OTP,EF,RETIRO OTP EXITOSO,24/02/2022 21:38,NaN,500000.00,24/02/2022 21:38,NaN,,CC,NaN,,,,GELSA-PAGA-TODO,9924070.0,9924070.0,NaN
2,4,A0007800374,RECAUDOS,EF,RECAUDO EXITOSO,24/02/2022 21:55,NaN,38270.00,24/02/2022 21:55,NaN,85050875,EF,NaN,0000157,,,GELSA-PAGA-TODO,88207611.0,88207611.0,NaN
3,4,A0007800398,RECAUDOS,EF,RECAUDO EXITOSO,25/02/2022 06:09,NaN,9390.00,25/02/2022 06:09,NaN,87924158,EF,NaN,0000157,,,GELSA-PAGA-TODO,9960490.0,9960490.0,NaN
4,4,A0007800404,RECAUDOS,EF,RECAUDO EXITOSO,25/02/2022 06:13,NaN,39780.00,25/02/2022 06:13,NaN,82595510,EF,NaN,0000157,,,GELSA-PAGA-TODO,9970983.0,9970983.0,NaN


In [23]:
frame.shape

(25782, 20)

In [24]:
frame['TIPO'].unique().tolist()

['RETIRO_OTP',
 'RECAUDOS',
 'PAGO_PRESTAMOS',
 'DEPOSITOS',
 'PAGO_TARJETA_CREDITO',
 'RECAUDO',
 'RETIROCB',
 nan]

In [25]:
frame['ESTADO'].unique().tolist()

['RETIRO OTP EXITOSO',
 'RECAUDO EXITOSO',
 'REVERSO RECAUDO EXITOSO',
 'RETIRO OTP NO EXITOSO',
 'PAGO PRESTAMOS EXITOSO',
 'DEPOSITO EXITOSO',
 'PAGO TARCRED EXITOSO',
 'PAGO PRESTAMOS NO EXITOSO',
 'PAGO TARCRED NO EXITOSO',
 'REVERSO RETIRO OTP EXITOSO',
 'PENDIENTEVALIDACION',
 'RETIROCBPAGADO',
 'TRANS-FAIL-C',
 'CARGOGENERADO',
 'FACTURAVALIDADA',
 'RETIROCBVALIDO',
 'CARGOFALLIDO',
 nan]

In [26]:
frame['CORRESPONSAL'].unique().tolist()

['GELSA-PAGA-TODO',
 'BALOTO',
 'JERONIMO MARTINS',
 'SUPERGIROS',
 'JJPITA',
 nan,
 'TRANZA',
 'FULLCARGA']

In [27]:
frame.groupby(['CORRESPONSAL','ESTADO']).size()

CORRESPONSAL      ESTADO                    
BALOTO            DEPOSITO EXITOSO              1509
                  PAGO PRESTAMOS EXITOSO         317
                  PAGO PRESTAMOS NO EXITOSO        4
                  PAGO TARCRED EXITOSO           518
                  PAGO TARCRED NO EXITOSO          3
                  RETIRO OTP EXITOSO            3258
                  RETIRO OTP NO EXITOSO           83
FULLCARGA         CARGOFALLIDO                     3
                  CARGOGENERADO                  589
                  FACTURAVALIDADA                 57
                  RETIROCBPAGADO                  42
                  RETIROCBVALIDO                   7
                  TRANS-FAIL-C                     1
GELSA-PAGA-TODO   RECAUDO EXITOSO               5238
                  RETIRO OTP EXITOSO             313
                  RETIRO OTP NO EXITOSO            7
                  REVERSO RECAUDO EXITOSO        100
JERONIMO MARTINS  RETIRO OTP EXITOSO            1288
 

<h3>Suma de Valores</h3>

In [28]:
frame['VALOR'] = frame['VALOR'].astype('float64')

In [29]:
print(frame.dtypes)

CB               object
UPC              object
TIPO             object
MEDIO            object
ESTADO           object
FECHA_OPER       object
FECHA_CONTA      object
VALOR           float64
FECHA_CREA       object
FECHA_CADUC      object
FACTURA          object
TIPO_CUENTA      object
NUM_CUENTA      float64
CONVENIO         object
MOVIMIENTO       object
MOVIMIENTO_2     object
CORRESPONSAL     object
PUNTO           float64
TERMINAL        float64
DESCRIP         float64
dtype: object


In [30]:
sum_success  = frame[frame['ESTADO'].isin(SUCCESS_ESTATE)].sum(axis=0)
#t_failure  = frame[frame['ESTADO'].isin(FAIL_ESTATE)].shape[0]
#t_others   = frame[~frame['ESTADO'].isin((SUCCESS_ESTATE + FAIL_ESTATE))].shape[0]

/var/folders/gh/blc43pl93nd97ctnhl_38w5m0000gn/T/ipykernel_12233/386225710.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  sum_success  = frame[frame['ESTADO'].isin(SUCCESS_ESTATE)].sum(axis=0)


In [31]:
sum_success['VALOR']

4923200576.35

In [32]:
output_data = [
    ["Total Transacciones",t_success],
    ["",],
    ["",],
    ["",],
    ["",],
    ["",],
    ["",],
    ["",],
    ["",]
]
